In [1]:
import os

In [2]:
os.chdir('/home/jennifer.badolato/watttime-python-client-aer-algo/')

In [33]:
import optimizer.s3 as s3u

In [3]:
import numpy as np
import pandas as pd
import datetime
import random
import pytz
from datetime import datetime, timedelta
import random
from watttime import WattTimeMyAccess, WattTimeHistorical, WattTimeForecast, WattTimeMaps
import os
import optimizer.dataset as od
from evaluation.config import MOER_REGION_LIST
import optimizer.s3 as s3u

username = os.getenv("WATTTIME_USER")
password = os.getenv("WATTTIME_PASSWORD")

start = '2024-02-15 00:00Z'
end = '2024-02-20 00:00Z'

distinct_date_list = [ pd.Timestamp(date).replace(tzinfo=pytz.UTC) for date in pd.date_range(start,end,freq='d', tz=pytz.UTC).values]

In [10]:
def generate_random_unplug_time(random_plug_time,mean,stddev):
    """ 
    Adds a number of sconds drawn from a normal distribution to the given datetime.

    Parameters:
    -datetime_obj
    -mean
    -stddev

    REturns 
    -pd.Timestamp: the new datetime after adding the random seconds
    """
    random_seconds = np.random.normal(loc=mean,scale=stddev)

    #convert to timedelta
    random_timedelta = timedelta(seconds=random_seconds)
    new_datetime = random_plug_time + random_timedelta

    if not isinstance(new_datetime,pd.Timestamp):
        new_datetime = pd.Timestamp(new_datetime)
    return new_datetime


def generate_random_plug_time(date):
    """
    Generate a random datetime ona the given date, uniformly distributed between 5pm and 9 pm.

    Parameters:
    date (datetime.date): The date on which to generate the random time.

    Returns:
    - datetime: A datetime object for the given date with a random time between 5 PM and 9 PM
    """
    #  Define the start and end times for the interval (5 PM to 9PM)
    start_time = datetime.combine(date, datetime.strptime("17:00:00", "%H:%M:%S").time())
    end_time = datetime.combine(date, datetime.strptime("21:00:00", "%H:%M:%S").time())
    
    # Calculate the total number of seconds between start and end times 
    total_seconds = int((end_time - start_time).total_seconds())
    
    # Generate a random number of seconds within the interval
    total_seconds = random.randint(0, total_seconds)

    # Add the random seconds to the start time to get the random datetime
    random_datetime = start_time + timedelta(seconds=total_seconds)

    random_datetime_utc = pytz.utc.localize(random_datetime)

    return random_datetime_utc

In [ ]:
#s3.store_csvdataframe(dataframe = df_hist, file="20240215_20240220_FPL_GVL_IID_IPCO_ISONE_CT.csv")

In [6]:
user_df = pd.DataFrame(
    distinct_date_list, 
    columns=['distinct_dates']
    ).sort_values(by='distinct_dates').copy()

In [7]:
user_df.head()

,distinct_dates
0,2024-02-15 00:00:00+00:00
1,2024-02-16 00:00:00+00:00
2,2024-02-17 00:00:00+00:00
3,2024-02-18 00:00:00+00:00
4,2024-02-19 00:00:00+00:00


In [8]:
rate = random.choice([11,7.4,22])
total_capacity =  round(random.uniform(21,123))
mean_length_charge = round(random.uniform(20000,30000))
std_length_charge = round(random.uniform(6800,8000))

In [9]:
user_df['user_type'] =  'r'+str(rate)+'_tc'+str(total_capacity)+'_avglc'+str(mean_length_charge)+'_sdlc'+str(std_length_charge)

In [23]:
user_df['plug_in_time'] = user_df['distinct_dates'].apply(generate_random_plug_time)

In [24]:
user_df['unplug_time'] = user_df['plug_in_time'].apply(lambda x: generate_random_unplug_time(x,mean_length_charge,std_length_charge))

In [25]:
# Another random parameter, this time at the session level, it's the initial charge of the battery.
user_df['initial_charge'] = user_df.apply(lambda _: random.uniform(0.2,0.6), axis=1)
user_df['total_seconds_to_95'] = user_df['initial_charge'].apply(
    lambda x: total_capacity*(0.95-x)/ rate * 3600
)

In [26]:
user_df['full_charge_time']= user_df['plug_in_time'] + pd.to_timedelta(user_df['total_seconds_to_95'],unit='s')
user_df['length_plugged_in'] = (user_df.unplug_time - user_df.plug_in_time) / pd.Timedelta(seconds=1)

In [30]:
# what happened first? did the user unplug or did it reach 95%
# rate / 3600 = rate per second
user_df['session_charge'] = user_df[['total_seconds_to_95','length_plugged_in']].min(axis=1)*(rate/3600)
user_df['final_perc_charged'] = user_df.session_charge.apply(lambda x: x/total_capacity)
user_df['final_perc_charged'] = user_df.final_perc_charged + user_df.initial_charge

In [38]:
# when did the device stop pulling power from the grid?
user_df['final_charge_time'] = user_df[['full_charge_time', 'unplug_time']].min(axis=1)
user_df['uncharged'] = np.where(user_df['final_perc_charged'] <0.80, True, False)

In [39]:
user_df.head().T

,0,1,2,3,4
distinct_dates,2024-02-15 00:00:00+00:00,2024-02-16 00:00:00+00:00,2024-02-17 00:00:00+00:00,2024-02-18 00:00:00+00:00,2024-02-19 00:00:00+00:00
user_type,r22_tc21_avglc28700_sdlc7501,r22_tc21_avglc28700_sdlc7501,r22_tc21_avglc28700_sdlc7501,r22_tc21_avglc28700_sdlc7501,r22_tc21_avglc28700_sdlc7501
initial_charge,0.308444,0.345806,0.275426,0.383768,0.354308
total_seconds_to_95,2204.618326,2076.23029,2318.08059,1945.780211,2047.014149
plug_in_time,2024-02-15 17:08:49+00:00,2024-02-16 18:49:22+00:00,2024-02-17 18:12:49+00:00,2024-02-18 17:33:16+00:00,2024-02-19 18:27:54+00:00
unplug_time,2024-02-15 23:08:44.620821+00:00,2024-02-17 04:24:48.488200+00:00,2024-02-17 21:55:10.352057+00:00,2024-02-19 05:02:28.161292+00:00,2024-02-20 02:06:01.734903+00:00
full_charge_time,2024-02-15 17:45:33.618326149+00:00,2024-02-16 19:23:58.230289866+00:00,2024-02-17 18:51:27.080590052+00:00,2024-02-18 18:05:41.780210697+00:00,2024-02-19 19:02:01.014148502+00:00
length_plugged_in,21595.620821,34526.4882,13341.352057,41352.161292,27487.734903
session_charge,13.472668,12.688074,14.166048,11.890879,12.509531
final_perc_charged,0.95,0.95,0.95,0.95,0.95
